# Pydantic Fundamentals

## validate_call

In [5]:
# Try without and with python type hints
def soma(a, b):
    return a + b

print(soma(1, 2))

def soma(a: int, b: int) -> int:
    return a + b

print(soma(2, 2))
print(soma("som", "as"))

from pydantic import validate_call

@validate_call
def soma(a: int, b: int) -> int:
    return a + b

print(soma(2, 2))
print(soma(1, "as"))

3
4
somas
4


ValidationError: 1 validation error for soma
1
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='as', input_type=str]
    For further information visit https://errors.pydantic.dev/2.1/v/int_parsing

In [9]:
from typing import Union
from pydantic import validate_call

def soma(a: int, b: Union[int, str]):
    return a + b

print(soma(1, 2))
print(soma('1', '3'))

@validate_call
def soma(a: Union[int, str], b: Union[int, str]):
    return a + b

print(soma(1, 2))
print(soma('1', '3'))

3
13
3
13


## Data Classes

In [22]:
from dataclasses import dataclass

@dataclass
class Person:
    name: str
    age: int

# This class dont have any validation
p1 = Person(name='John', age='42')
print(p1)

from pydantic.dataclasses import dataclass as pydantic_dataclass

@pydantic_dataclass
class PersonPydantic:
    name: str
    age: int


# This class have validation and will change str to int
p2 = PersonPydantic(name='John', age='42')
print(p2)


# Can use strictInt to only receive int and DONT change str to int
from pydantic import StrictInt

@pydantic_dataclass
class PersonPydanticStrict:
    name: str
    age: StrictInt

p3 = PersonPydanticStrict(name='John', age='42')

Person(name='John', age='42')
PersonPydantic(name='John', age=42)


ValidationError: 1 validation error for PersonPydanticStrict
age
  Input should be a valid integer [type=int_type, input_value='42', input_type=str]
    For further information visit https://errors.pydantic.dev/2.1/v/int_type

## Base Models

In [10]:
from pydantic import BaseModel

class Register(BaseModel):
    name: str
    age: int
    email: str
    password: str
    is_active: bool = True

# Creating a new register
r = Register(name='John', age=20, email="admin@admin.com", password="123456", is_active=False)
r_2 = Register(**{'name': 'Arthur', 'age': 21, 'email': 'email@email.com', 'password':'12'})

print(r)
print(r_2)

print(r.name)
print(r.age)
print(r_2.name)
print(r_2.age)

class Registers(BaseModel):
    registers: list[Register]

reg_list = [
    {'name': 'Arthur', 'age': 21, 'email': 'email@email.com', 'password':'12'},
    {'name': 'Joao', 'age': 23, 'email': 'email@email2.com', 'password':'32'}
]

# Adding this registers in Registers Class trought a list of dicts
regs = Registers(registers=reg_list)
print(regs)
print(regs.registers[0].name)
print(regs.registers[1].name)


name='John' age=20 email='admin@admin.com' password='123456' is_active=False
name='Arthur' age=21 email='email@email.com' password='12' is_active=True
John
20
Arthur
21
registers=[Register(name='Arthur', age=21, email='email@email.com', password='12', is_active=True), Register(name='Joao', age=23, email='email@email2.com', password='32', is_active=True)]
Arthur
Joao


In [55]:
# Validating fields
from pydantic import BaseModel, field_validator, model_validator

class Register(BaseModel):
    email: str
    password1: str
    password2: str

    @field_validator('email')
    def email_must_contain_at(cls, value):
        if '@' not in value:
            raise ValueError('must contain @')
        return value

    @field_validator('password1', 'password2')
    def passwords_must_be_longer_than_8_char(cls, value):
        if len(value) < 8:
            raise ValueError('Password must be at least 8 characters long')
        return value

    @model_validator(mode='after')
    def passwords_match(self):
        pw1 = self.password1
        pw2 = self.password2
        if pw1 != pw2:
            raise ValueError('passwords do not match')
        return self

# right value
r1 = Register(email='aaaa@addd.com', password1='123456789', password2='123456789')
# wrong email 
r2 = Register(email='samuel', password1='1234', password2='12345') 



ValidationError: 3 validation errors for Register
email
  Value error, must contain @ [type=value_error, input_value='samuel', input_type=str]
    For further information visit https://errors.pydantic.dev/2.1/v/value_error
password1
  Value error, Password must be at least 8 characters long [type=value_error, input_value='1234', input_type=str]
    For further information visit https://errors.pydantic.dev/2.1/v/value_error
password2
  Value error, Password must be at least 8 characters long [type=value_error, input_value='12345', input_type=str]
    For further information visit https://errors.pydantic.dev/2.1/v/value_error

In [61]:
# Pre validator

from pydantic import BaseModel, field_validator

class Orders(BaseModel):
    ids: list[int]

    @field_validator('ids', mode='before')
    def convert_ids(cls, v):
        if type(v) is str:
            return [int(i) for i in v.split(',')]
        return v

print(Orders(ids='1,2,3'))
print(Orders(ids=[3,2,1]))



ids=[1, 2, 3]
ids=[3, 2, 1]


In [71]:
# Pydantic Fields

from pydantic import BaseModel, EmailStr

class User(BaseModel):
    name: str
    email: EmailStr
    password: str

print(User(name='John Doe', email='abcde@gmail.com', password='123456'))
print(User(name='John Doe', email='abcde@gmailom', password='123456'))

name='John Doe' email='abcde@gmail.com' password='123456'


ValidationError: 1 validation error for User
email
  value is not a valid email address: The part after the @-sign is not valid. It should have a period. [type=value_error, input_value='abcde@gmailom', input_type=str]